# Weighted Cost Function

Shows how to use the cost function requested in [issue #84](https://github.com/EthanJamesLew/AutoKoopman/issues/84).

In [ ]:
# the notebook imports
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append("..")
# this is the convenience function
from autokoopman import auto_koopman

In [ ]:
# for a complete example, let's create an example dataset using an included benchmark system
import autokoopman.benchmark.fhn as fhn
fhn = fhn.FitzHughNagumo()
training_data = fhn.solve_ivps(
    initial_states=np.random.uniform(low=-2.0, high=2.0, size=(10, 2)),
    tspan=[0.0, 6.0],
    sampling_period=0.1
)

In [ ]:
# create trajectories as numpy array and create a weights array
# NOTE: learning_weights does not allow you to weight state, but the observations!
trajectories = []
weights = []

# create weights for every time point
for idx, traj in enumerate(training_data):
    # good trajectory
    trajectories.append(traj.states)

    # garbage trajectory
    trajectories.append(np.random.rand(*traj.states.shape))
    
    # weight good trajectory by its 1 norm
    #w = np.sum(traj.abs().states, axis=1)
    #w = 1/(traj.abs().states+1.0)
    w = np.ones(traj.states.shape)
    w[:, 0] = 0.2
    weights.append(w)

    # weight garbage trajectory to zero
    #w = np.zeros(len(traj.states))
    w = np.zeros(traj.states.shape)
    weights.append(w)

# you can also use a dict to name the trajectories if using TrajectoriesData (numpy arrays are named by their index number)
#weights = {idx: w for idx, w in enumerate(weights)}

In [ ]:
weights[0].shape

In [ ]:
# learn model from weighted data
experiment_results = auto_koopman(
    trajectories,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type="rff",         # use Random Fourier Features Observables
    cost_func="weighted",   # use "weighted" cost function
    learning_weights=weights, # weight the eDMD algorithm objectives
    scoring_weights=weights, # pass weights as required for cost_func="weighted"
    opt="grid",             # grid search to find best hyperparameters
    n_obs=30,              # maximum number of observables to try
    max_opt_iter=200,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# learn unweighted model from data
experiment_results_unweighted = auto_koopman(
    trajectories,          # list of trajectories
    sampling_period=0.1,    # sampling period of trajectory snapshots
    obs_type="rff",         # use Random Fourier Features Observables
    cost_func="weighted",   # use "weighted" cost function
    learning_weights=None,  # don't use eDMD weighting
    scoring_weights=weights, # pass weights as required for cost_func="weighted"
    opt="grid",             # grid search to find best hyperparameters
    n_obs=30,              # maximum number of observables to try
    max_opt_iter=200,       # maximum number of optimization iterations
    grid_param_slices=5,   # for grid search, number of slices for each parameter
    n_splits=5,             # k-folds validation for tuning, helps stabilize the scoring
    rank=(1, 200, 40)       # rank range (start, stop, step) DMD hyperparameter
)

In [ ]:
# view our custom weighted cost
experiment_results

In [ ]:
# get the model from the experiment results
model = experiment_results['tuned_model']
model_uw = experiment_results_unweighted['tuned_model']

# simulate using the learned model
iv = [0.1, 0.5]
trajectory = model.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)
trajectory_uw = model_uw.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)

In [ ]:
# simulate the ground truth for comparison
true_trajectory = fhn.solve_ivp(
    initial_state=iv,
    tspan=(0.0, 10.0),
    sampling_period=0.1
)

plt.figure(figsize=(10, 6))

# plot the results
plt.plot(*true_trajectory.states.T, linewidth=2, label='Ground Truth')
plt.plot(*trajectory.states.T, label='Weighted Trajectory Prediction')
plt.plot(*trajectory_uw.states.T, label='Trajectory Prediction')


plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.grid()
plt.legend()
plt.title("FHN Test Trajectory Plot")
plt.show()

In [ ]:
plt.plot(true_trajectory.states[:, 0], linewidth=2, label='Ground Truth')
plt.plot(trajectory.states[:, 0], label='Weighted Trajectory Prediction')
plt.plot(trajectory_uw.states[:, 0], label='Trajectory Prediction')

In [ ]:
plt.plot(true_trajectory.states[:, 1], linewidth=2, label='Ground Truth')
plt.plot(trajectory.states[:, 1], label='Weighted Trajectory Prediction')
plt.plot(trajectory_uw.states[:, 1], label='Trajectory Prediction')

In [ ]:
from casadi import *


In [ ]:
X = np.vstack([t.states[:-1] for t in training_data])
Xp = np.vstack([t.states[1:] for t in training_data])
W = [w[:-1] for w in weights]
g = experiment_results['tuned_model'].obs_func
gd = experiment_results['tuned_model'].obs.obs_grad
G = np.vstack([np.atleast_2d(g(x)).T for x in X])
Gp = np.vstack([np.atleast_2d(g(x)).T for x in Xp])

G, Gd = G.T, Gp.T
Js = [gd(xi) for xi in X]
Wy = np.vstack(
        [(np.abs(J) @ w.T).T 
         for J, w in zip(Js, W)
    ])

In [ ]:
(Js[0] @ W[0].T).shape

In [ ]:
Wy.T * G

In [ ]:
J.shape